In [ ]:
# install the follwoing packages in the enviroment:
# python3 -m pip install pandas
# python3 -m pip install seaborn

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import json

import os

## Hypotheses to be tested

#### As reference values to compare to, we will use demographics from ECS from agu, egu, ssa. This will give an idea of how many of the active scientists at these professional levels are represented to the peer-reviewed articles (which is the main crucial factor for career advancing and perhaps the daily goal of most academics)

FIRST GLANCES AT DATA

- % of female first authors (hists?)
- % publications with all male vs. % publications with all female authors (hists?)

CO-AUTHORSHIP ANALYSES

- When 1st author is female: % of male vs. female co-authors (bars..?)
- When 1st author is female: likelihood of last author (possibly PI) to be female vs. male 
- When 1st author is male: % of female coauthors and % of male coauthors
- When the last name is female (possible PI), is there a higher % of female co-authors vs. male ones?

JOURNAL IF ANALYSES:

- Correlation between IF and female first authors: does higher IF mean fewer female first authors?




In [ ]:
# Define local paths

root = ! pwd
root = root[0]

RAW_DIR=root+"/author_allgenders/"  # TODO: change this with the final json files obtained from parsing

if not os.path.exists(RAW_DIR):
    print("The directory {} does not exist.\nThere is no raw data for statistical analysis.".format(RAW_DIR))

In [ ]:
## read all json files and create a dataframe

dfs = [] 

for _, _, files in os.walk(RAW_DIR):
    for file in files:
        with open(RAW_DIR+file) as f:
            data = json.loads(f.read())
            df = pd.json_normalize(data)
            df = df.drop(columns=['first_author','all_names', 'year', 'month', 'title', 'id'])
            dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df